In [1]:
import glob

import faiss
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Load models

In [2]:
llm = ChatOllama(model="llama3.2:latest", temperature=0)
embeddings_model = OllamaEmbeddings(model="mxbai-embed-large:latest")

## Load test pdf files

In [3]:
pdf_paths = glob.glob("test-data/*.pdf")

pages = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    async for page in loader.alazy_load():
        pages.append(page)

Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 75 0 (offset 0)
Ignoring wrong pointing object 90 0 (offset 0)
Ignoring wrong pointing object 115 0 (offset 0)


## Load test txt files

In [4]:
loader = DirectoryLoader(path="test-data", glob="*.txt", loader_cls=TextLoader)
pages = pages + loader.load()

## Split data in chunks

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(pages)

## Generate embeddings and save them in vector store

In [6]:
index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))  #Get dimensions of embeddings

vectorstore = FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

vectorstore.add_documents(documents=splits)

retriever = vectorstore.as_retriever(k=4)

## Make RAG request

In [12]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. Answer only with the information and not with any kind of chat!"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

question = "Warum sollte ich Daten kapseln?"

answer = rag_chain.invoke({"input": question})

print(f"Frage: {answer['input']}")
print(f"Antwort: {answer['answer']}")
print()
print("Quellen:")
for source in answer["context"]:
    print(f"\t{source.metadata['source']} - Seite: {source.metadata['page']}")

Frage: Warum sollte ich Daten kapseln?
Antwort: Du solltest Datenkapselung verwenden, um deine Programmkomponenten losgelöst zu machen und Abhängigkeiten zwischen ihnen zu reduzieren. Dadurch kannst du die Sicherheit deiner Daten erhöhen, indem du sie vor unbefugtem Zugriff versteckst.
Quellen:
	test-data\Prog1_K06.pdf - Seite: 25
	test-data\Prog1_K06.pdf - Seite: 0
	test-data\Prog1_K06.pdf - Seite: 6
	test-data\Prog1_K06.pdf - Seite: 7
